### Test case part 1: calculating H_nn for a bath with 2 nuclei with I=0.5. 

For two nuclei: we use the following basis:

$\begin{bmatrix}
\uparrow \uparrow
\\ \uparrow \downarrow
\\ \downarrow \uparrow
\\ \downarrow \downarrow
\end{bmatrix}$

Evidently, $\hat{I}_z$ matrix takes the form of 

$$\hat{I_z} =  \hat{\sigma}_z \otimes \mathbb{1}  + \mathbb{1} \otimes \hat{\sigma}_z,$$

$$\sigma_z = \begin{bmatrix}
1 & 0\\
0 & -1 
\end{bmatrix}$$

or 

$$ \hat{I}_z = 
\begin{bmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & -1 & 0\\
0 & 0 & 0 & -1\\
\end{bmatrix} + 
\begin{bmatrix}
1 & 0 & 0 & 0\\
0 & -1 & 0 & 0\\
0 & 0 & 1 & 0\\
0 & 0 & 0 & -1\\
\end{bmatrix} = \begin{bmatrix}
2 & 0 & 0 & 0\\
0 & 0 & 0 & 0\\
0 & 0 & 0 & 0\\
0 & 0 & 0 & -2\\
\end{bmatrix}
$$

For general nuclear spin operator in 3D: 
$$\hat{I_1} = \vec{\sigma} \otimes \mathbb{1}$$
$$\hat{I_2} = \mathbb{1} \otimes \vec{\sigma},$$

and

$$\vec{\sigma} = \hat{\sigma}_x + \hat{\sigma}_y + \hat{\sigma}_z$$

below I attempt to write the nuclear-nuclear interaction part of the bath hamiltonian (i,j sums over bath nuclei):

$$H_{nn} = \frac{\mu_0}{4\pi}\sum_{i<j}{}\gamma_i\gamma_j\hbar^2(\frac{\vec{I}_i\cdot\vec{I}_j}{r_{ij}^{3}} - \frac{3(\vec{I}_i \cdot \vec{r}_{ij})(\vec{I}_j \cdot \vec{r}_{ij})}{r_{ij}^{5}})$$

$\vec{r}_{ij}$ is the vector connecting nuclei i,j, $\gamma_{i,j}$ are the gyromagnetic ratio of nuclei i, and nuclei j.

In [17]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from qutip import *

B = np.array([0,0,1]) #magnetic field (unused now)
r1=[3,4,5] #position of n1
r2=r1+[1,3,4]/np.sqrt(3) #position of n2
r=r2-r1
c=1 #constant to be determined. includes all scalar terms
nuclear_spin=[0.5,0.5] # nuclear spin of spin 1 and 2


I1=np.array([tensor(spin_Jx(nuclear_spin[0]),qeye(2)), #use J spin matrix in case spin !=1/2 
             tensor(spin_Jy(nuclear_spin[0]),qeye(2)),
             tensor(spin_Jz(nuclear_spin[0]),qeye(2))], 
            dtype=object)
I2=np.array([tensor(qeye(2),spin_Jx(nuclear_spin[1])),
             tensor(qeye(2),spin_Jy(nuclear_spin[1])),
             tensor(qeye(2),spin_Jz(nuclear_spin[1]))],
            dtype=object)

H_nn = c*(I1@I2/np.linalg.norm(r)**3 - 3*(I1@r)*(I2@r)/np.linalg.norm(r)**5)

In [18]:
H_nn

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-0.00829108+0.j         -0.00452241+0.01356722j -0.00452241+0.01356722j
   0.00904482+0.00678361j]
 [-0.00452241-0.01356722j  0.00829108+0.j          0.00829108+0.j
   0.00452241-0.01356722j]
 [-0.00452241-0.01356722j  0.00829108+0.j          0.00829108+0.j
   0.00452241-0.01356722j]
 [ 0.00904482-0.00678361j  0.00452241+0.01356722j  0.00452241+0.01356722j
  -0.00829108+0.j        ]]

### Test case part 2: calculating nuclear Zeeman Hamiltonian

The nuclear zeeman hamiltonian is calculated using the following formula:

$$H_{n} = -\vec{B} \cdot \sum_{i}\gamma_i \hbar \vec{I}_i,$$

with  (i sums over bath nuclei).

In the example below I'm using $\gamma_{1,2} = 1,2$ to show case when nuclear gyromagnetic moments of nuclei 1,2 are not equal. Since magnetic field is along z, only $\hat{\sigma}_z$ is expected. The diagonal element of the zeeman hamiltonian corresponds to the expectation value of energy proportional to the expectation value of the spin along the z.

In [3]:
%matplotlib inline
import numpy as np
from qutip import *

B = np.array([0,0,1]) #magnetic field (unused now)
hbar=1 #constant
gamma1=1
gamma2=2
nuclear_spin=[0.5,0.5] # nuclear spin of spin 1 and 2


I1=np.array([tensor(spin_Jx(nuclear_spin[0]),qeye(2)), #use J spin matrix in case spin !=1/2 
             tensor(spin_Jy(nuclear_spin[0]),qeye(2)),
             tensor(spin_Jz(nuclear_spin[0]),qeye(2))], 
            dtype=object)
I2=np.array([tensor(qeye(2),spin_Jx(nuclear_spin[1])),
             tensor(qeye(2),spin_Jy(nuclear_spin[1])),
             tensor(qeye(2),spin_Jz(nuclear_spin[1]))],
            dtype=object)

H_n = -hbar* B@(np.array([gamma1,gamma2])@np.array([I1,I2])) # equivalent to ...@(gamma1*I1+gamma2*I2)

In [4]:
H_n

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-1.5  0.   0.   0. ]
 [ 0.   0.5  0.   0. ]
 [ 0.   0.  -0.5  0. ]
 [ 0.   0.   0.   1.5]]

### Test case part3: system bath interaction thru the hyperfine tensor

Taking hamiltonian with certain approximations from https://static-content.springer.com/esm/art%3A10.1038%2Fncomms12935/MediaObjects/41467_2016_BFncomms12935_MOESM89_ESM.pdf equation (9), (7):

$$H_{hyperfine} = m_s \sum_{i} \vec{A}_i \cdot \vec{I}_i = m_s \sum_{i} (A_{zx,i}I_{ix} + A_{zy,i}I_{iy} + A_{zz,i}I_{iz}),$$

with i sums over bath nuclei, $A_{zx}, A_{zy}$ are the pseudo-secular hyperfine couplings, $A_{zz}$ is the secular hyperfine coupling. 

For $m_s = \frac{1}{2}$:

In [5]:
%matplotlib inline
import numpy as np
from qutip import *

ms=1/2
I_array=[0.5,0.5] # nuclear spin of spin 1 and 2

I1=np.array([tensor(spin_Jx(nuclear_spin[0]),qeye(2)), #use J spin matrix in case spin !=1/2 
             tensor(spin_Jy(nuclear_spin[0]),qeye(2)),
             tensor(spin_Jz(nuclear_spin[0]),qeye(2))], 
            dtype=object)
I2=np.array([tensor(qeye(2),spin_Jx(nuclear_spin[1])),
             tensor(qeye(2),spin_Jy(nuclear_spin[1])),
             tensor(qeye(2),spin_Jz(nuclear_spin[1]))],
            dtype=object)


A_tensor_i = np.array([[1,2,3],[4,5,6]]) #dummy A tensor. row loops over nuclei, col over xz, yz, zz

H_hyperfine = ms*np.sum(A_tensor_i*np.array([I1,I2])) #equivalent to ms*(A_tensor_i[0,:]@I1+A_tensor_i[1,:]@I2)

In [6]:
H_hyperfine

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 2.25+0.j    1.  -1.25j  0.25-0.5j   0.  +0.j  ]
 [ 1.  +1.25j -0.75+0.j    0.  +0.j    0.25-0.5j ]
 [ 0.25+0.5j   0.  +0.j    0.75+0.j    1.  -1.25j]
 [ 0.  +0.j    0.25+0.5j   1.  +1.25j -2.25+0.j  ]]

### Putting everything together:
$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$
$\newcommand{\bra}[1]{\left\langle{#1}\right|}$

From https://static-content.springer.com/esm/art%3A10.1038%2Fncomms12935/MediaObjects/41467_2016_BFncomms12935_MOESM89_ESM.pdf equation (8), (9):

$$H_{total} = \sum_{m_s = -\frac{1}{2}}^{+\frac{1}{2}}\ket{m_s}\bra{m_s} \otimes H_{m_s} , $$

$$ H_{m_s} = \omega_{m_s} + H_{n} + H_{nn} + m_s\sum_{i} \vec{A}_i \cdot \vec{I}_i ,$$

with $\omega_{m_s} =  -\gamma_e \hbar B_z m_s * \mathbb{1}$, $H_{n}$ is the Bath's nuclear zeeman hamiltonian, $H_{nn}$ is the Bath's nuclear-nuclear interactions Hamiltonian.

In [20]:
gamma_e = 2 
hbar = 1
Bz = B[2]
ms = 1/2

omega_ms = -gamma_e *hbar *Bz *ms*tensor(qeye(2),qeye(2))

projector_up = basis(2,0)*dag(basis(2,0))
projector_down =  basis(2,1)*dag(basis(2,1))

H_tot = tensor(projector_up, (omega_ms+H_n+H_nn+H_hyperfine)) + tensor(projector_down, (-omega_ms+H_n+H_nn-H_hyperfine))

In [21]:
H_tot

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[-0.25 +0.j     0.998-1.248j  0.248-0.498j  0.   +0.004j  0.   +0.j
   0.   +0.j     0.   +0.j     0.   +0.j   ]
 [ 0.998+1.248j -1.25 +0.j     0.   +0.j     0.252-0.502j  0.   +0.j
   0.   +0.j     0.   +0.j     0.   +0.j   ]
 [ 0.248+0.498j  0.   +0.j    -0.75 +0.j     1.002-1.252j  0.   +0.j
   0.   +0.j     0.   +0.j     0.   +0.j   ]
 [ 0.   -0.004j  0.252+0.502j  1.002+1.252j -1.75 +0.j     0.   +0.j
   0.   +0.j     0.   +0.j     0.   +0.j   ]
 [ 0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j    -2.75 +0.j
  -1.002+1.252j -0.252+0.502j  0.   +0.004j]
 [ 0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j    -1.002-1.252j
   2.25 +0.j     0.   +0.j    -0.248+0.498j]
 [ 0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j    -0.252-0.502j
   0.   +0.j    -0.25 +0.j    -0.998+1.248j]
 [ 0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   -0.004j
  -0.248-0.498j -0.998

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from qutip import *

nuclear_spin=[0.5,0.5]

I1=np.array([tensor(spin_Jx(nuclear_spin[0]),qeye(2)), #use J spin matrix in case spin !=1/2 
             tensor(spin_Jy(nuclear_spin[0]),qeye(2)),
             tensor(spin_Jz(nuclear_spin[0]),qeye(2))], 
            dtype=object)

I2=np.array([tensor(qeye(2),spin_Jx(nuclear_spin[1])),
             tensor(qeye(2),spin_Jy(nuclear_spin[1])),
             tensor(qeye(2),spin_Jz(nuclear_spin[1]))],
            dtype=object)

In [5]:
I1_plus=tensor(create(2),qeye(2))
I1_plus

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]

In [6]:
I2_minus=tensor(qeye(2),destroy(2))
I2_minus

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]]

In [7]:
I1_plus*I2_minus

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]]

In [8]:
I1_minus=tensor(destroy(2),qeye(2))
I1_minus

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

In [9]:
I2_plus=tensor(qeye(2),create(2))
I2_plus

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]]

In [10]:
I1_minus*I2_plus

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

In [19]:
I1[2]*I2[2]

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.25  0.    0.    0.  ]
 [ 0.   -0.25  0.    0.  ]
 [ 0.    0.   -0.25  0.  ]
 [ 0.    0.    0.    0.25]]

In [20]:
tensor(spin_Jz(nuclear_spin[0]),qeye(2))

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.5  0.   0.   0. ]
 [ 0.   0.5  0.   0. ]
 [ 0.   0.  -0.5  0. ]
 [ 0.   0.   0.  -0.5]]

In [22]:
tensor(sigmaz(),qeye(2))

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0. -1.]]

In [23]:
sigmap()

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[0. 1.]
 [0. 0.]]

In [27]:
destroy(2)

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[0. 1.]
 [0. 0.]]

In [36]:
I1[0]-1.j*I1[1]

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]

In [35]:
tensor(create(2),qeye(2))

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]